In [1]:
import sys
sys.path.append("../")

In [17]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf

2023-04-13 02:14:35.208613: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Reading the CSV and images

In [3]:
dicom_dataset = pd.read_csv("../dataset/csv/dicom_info.csv")
mass_train= pd.read_csv("../dataset/csv/mass_case_description_train_set.csv")

In [4]:
dicom_dataset["PatientID"]

0        Mass-Training_P_01265_RIGHT_MLO_1
1           Mass-Training_P_01754_RIGHT_CC
2           Calc-Training_P_00232_RIGHT_CC
3              Calc-Test_P_00562_LEFT_CC_2
4                    P_00562_LEFT_CC_2.dcm
                       ...                
10232     Calc-Training_P_01864_RIGHT_CC_2
10233           Mass-Test_P_00979_LEFT_MLO
10234      Mass-Training_P_01461_RIGHT_MLO
10235    Mass-Training_P_00314_RIGHT_MLO_1
10236    Mass-Training_P_00314_RIGHT_MLO_1
Name: PatientID, Length: 10237, dtype: object

In [5]:
image_paths = mass_train["image file path"].values
patient_ids = []

for image_path in image_paths:
    path = os.path.normpath(image_path)
    patient_ids.append(path.split(os.sep)[0])


In [6]:
mass_train["PatientID"] = patient_ids

In [7]:
mass_train = mass_train.merge(dicom_dataset[["PatientID", "image_path"]], on='PatientID', how='inner')

In [8]:
image_paths = mass_train["image_path"].values
image_cropped_paths = []

for image_path in image_paths:
    path = os.path.normpath(image_path)
    x = path.split(os.sep)
    x[0] = "../dataset"
    image_cropped_paths.append(os.path.join(*x))

In [9]:
mass_train["fixed_image_paths"] = image_cropped_paths

In [10]:
mass_train.head()

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,mass shape,mass margins,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path,PatientID,image_path,fixed_image_paths
0,P_00001,3,LEFT,CC,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,Mass-Training_P_00001_LEFT_CC/1.3.6.1.4.1.9590...,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00001_LEFT_CC,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.342386...,../dataset/jpeg/1.3.6.1.4.1.9590.100.1.2.34238...
1,P_00001,3,LEFT,MLO,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,Mass-Training_P_00001_LEFT_MLO/1.3.6.1.4.1.959...,Mass-Training_P_00001_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00001_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00001_LEFT_MLO,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.359308...,../dataset/jpeg/1.3.6.1.4.1.9590.100.1.2.35930...
2,P_00004,3,LEFT,CC,1,mass,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,Mass-Training_P_00004_LEFT_CC/1.3.6.1.4.1.9590...,Mass-Training_P_00004_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00004_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00004_LEFT_CC,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.891800...,../dataset/jpeg/1.3.6.1.4.1.9590.100.1.2.89180...
3,P_00004,3,LEFT,MLO,1,mass,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,Mass-Training_P_00004_LEFT_MLO/1.3.6.1.4.1.959...,Mass-Training_P_00004_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00004_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00004_LEFT_MLO,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.295360...,../dataset/jpeg/1.3.6.1.4.1.9590.100.1.2.29536...
4,P_00004,3,RIGHT,MLO,1,mass,OVAL,CIRCUMSCRIBED,4,BENIGN,5,Mass-Training_P_00004_RIGHT_MLO/1.3.6.1.4.1.95...,Mass-Training_P_00004_RIGHT_MLO_1/1.3.6.1.4.1....,Mass-Training_P_00004_RIGHT_MLO_1/1.3.6.1.4.1....,Mass-Training_P_00004_RIGHT_MLO,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.410524...,../dataset/jpeg/1.3.6.1.4.1.9590.100.1.2.41052...


In [11]:
train_dataset = mass_train[["pathology", "fixed_image_paths"]]

In [13]:
len(train_dataset)

1318

In [15]:
images = []
for i, image_path in enumerate(train_dataset["fixed_image_paths"].values):
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    images.append(image)

KeyboardInterrupt: 

In [19]:
model = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=2,
    classifier_activation='softmax'
)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-04-13 02:15:32.276250: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-13 02:15:32.276856: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


87910968/87910968 [==============================] - 13s 0us/step


In [21]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   96         ['conv2d[0][0]']                 
 alization)                     32)                                                    

In [8]:
# Load multiple scan images from directory
dataset_dir = '../dataset/jpeg/'
image_list=[]
folder_name_list = os.listdir(dataset_dir)[:100]
for foldername in folder_name_list:
    image_dir = dataset_dir + foldername
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            image = cv2.imread(os.path.join(image_dir, filename))
            if image is not None:
                image_list.append(image)


In [92]:
if not os.path.exists("../dataset/jpeg/train"):
    os.makedirs("../dataset/jpeg/train")
if not os.path.exists("../dataset/jpeg/valid"):
    os.makedirs("../dataset/jpeg/valid")
if not os.path.exists("../dataset/jpeg/test"):
    os.makedirs("../dataset/jpeg/test")



In [103]:
import numpy as np
import cv2
import os
import pydicom
from sklearn.model_selection import train_test_split

# Load the dataset
data_dir = '../dataset/jpeg'
image_size = 224
images = []
labels = []
for foldername in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, foldername)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        dcm = pydicom.dcmread(file_path, force=True)
        image = dcm.pixel_array.astype(float)
        image = cv2.resize(image, (image_size, image_size))
        image = image / 255.0
        images.append(image)
        if foldername == 'benign':
            labels.append(0)
        elif foldername == 'malignant':
            labels.append(1)

# Split the dataset
X_trainval, X_test, y_trainval, y_test = train_test_split(images, labels, test_size=0.15, random_state=42, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.18, random_state=42, stratify=y_trainval)

# Save the splits to disk
np.savez('dataset_splits.npz', X_train=X_train, X_val=X_val, X_test=X_test, y_train=y_train, y_val=y_val, y_test=y_test)


AttributeError: 'FileMetaDataset' object has no attribute 'TransferSyntaxUID'

In [99]:
import os
import random
import shutil

import cv2
import numpy as np

# Define paths to the DICOM dataset and output directories
dicom_path = "../dataset/jpeg"
output_path = "../output"

# Define paths to train and test directories
train_path = os.path.join(output_path, "train")
test_path = os.path.join(output_path, "test")

# Create output directories if they don't exist
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

# Define parameters for image resizing and normalization
image_size = 224

# Get list of DICOM files
files = os.listdir(dicom_path)
random.shuffle(files)

# Split files into train and test sets
split = int(0.8 * len(files))
train_files = files[:split]
test_files = files[split:]

# Copy train files to train directory
for filename in train_files:
    shutil.copy(os.path.join(dicom_path, filename), os.path.join(train_path, filename))

# Copy test files to test directory
for filename in test_files:
    shutil.copy(os.path.join(dicom_path, filename), os.path.join(test_path, filename))

# Load train images
train_images = []
for filename in train_files:
    file_path = os.path.join(train_path, filename)
    dcm = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
    image = cv2.resize(dcm, (image_size, image_size))
    image = np.float32(image) / 65535.0  # Normalize pixel values to [0, 1]
    train_images.append(image)

# Load test images
test_images = []
for filename in test_files:
    file_path = os.path.join(test_path, filename)
    dcm = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
    image = cv2.resize(dcm, (image_size, image_size))
    image = np.float32(image) / 65535.0  # Normalize pixel values to [0, 1]
    test_images.append(image)

# Convert lists to numpy arrays
train_images = np.array(train_images)
test_images = np.array(test_images)

# Print the number of images in each set
print("Number of training images:", len(train_images))
print("Number of testing images:", len(test_images))


IsADirectoryError: [Errno 21] Is a directory: '../dataset/jpeg/1.3.6.1.4.1.9590.100.1.2.383106931112571205227813146982972889681'

In [95]:

print(len(train_images))



0
